In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import keras

#from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
path = '/Users/sanjayagrawal/Downloads/Kaggle/ship/'

In [7]:
#Import Keras.
#from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [8]:
from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=64

In [3]:
train = pd.read_json(path + "train.json")
target_train=train['is_iceberg']
test = pd.read_json(path +"test.json")

In [4]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [5]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
#X_band_3=(X_band_1+X_band_2)/2
X_band_3=np.fabs(np.subtract(X_band_1,X_band_2))
X_band_4=np.maximum(X_band_1,X_band_2)
X_band_5=np.minimum(X_band_1,X_band_2)
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([
                          
                          X_band_3[:, :, :, np.newaxis],X_band_4[:, :, :, np.newaxis],X_band_5[:, :, :, np.newaxis]], axis=-1)

In [6]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
#X_band_test_3=(X_band_test_1+X_band_test_2)/2
X_band_test_3=np.fabs(np.subtract(X_band_test_1,X_band_test_2))
X_band_test_4=np.maximum(X_band_test_1,X_band_test_2)
X_band_test_5=np.minimum(X_band_test_1,X_band_test_2)
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([
                          X_band_test_3[:, :, :, np.newaxis], X_band_test_4[:, :, :, np.newaxis],X_band_test_5[:, :, :, np.newaxis]],axis=-1)

In [9]:
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.5,
                         rotation_range = 10)

In [10]:
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

In [12]:
# Finally create generator
def get_callbacks(filepath, patience=2):
   #es = EarlyStopping('val_loss', patience=10, mode="min")
   es = EarlyStopping('val_loss', patience=20, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]

In [13]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    x = GlobalMaxPooling2D()(x)
    base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

    x2 = base_model2.output
    x2 = GlobalAveragePooling2D()(x2)

    merge_one = concatenate([x, x2, angle_layer])

    merge_one = Dropout(0.6)(merge_one)
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [14]:
def myAngleCV(X_train, X_angle, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=10)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [15]:
preds=myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0
58892288/58889256 [==============================] - 96s 2us/step


/Users/sanjayagrawal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - 677s 28s/step - loss: 0.9498 - acc: 0.5243 - val_loss: 0.8232 - val_acc: 0.4689
Epoch 2/100
24/24 [==============================] - 644s 27s/step - loss: 0.6908 - acc: 0.5874 - val_loss: 0.7329 - val_acc: 0.4689
Epoch 3/100
24/24 [==============================] - 656s 27s/step - loss: 0.6767 - acc: 0.5763 - val_loss: 0.7461 - val_acc: 0.4689
Epoch 4/100
24/24 [==============================] - 652s 27s/step - loss: 0.7060 - acc: 0.5652 - val_loss: 0.7686 - val_acc: 0.4689
Epoch 5/100
24/24 [==============================] - 585s 24s/step - loss: 0.6825 - acc: 0.5698 - val_loss: 0.7824 - val_acc: 0.4689
Epoch 6/100
24/24 [==============================] - 609s 25s/step - loss: 0.6634 - acc: 0.5789 - val_loss: 0.7975 - val_acc: 0.4689
Epoch 7/100
24/24 [==============================] - 605s 25s/step - loss: 0.6569 - acc: 0.6230 - val_loss: 0.8034 - val_acc: 0.4689
Epoch 8/100
24/24 [==============================] - 646s 27s/step - 

24/24 [==============================] - 592s 25s/step - loss: 0.5114 - acc: 0.7170 - val_loss: 0.4014 - val_acc: 0.8442
Epoch 9/100
24/24 [==============================] - 600s 25s/step - loss: 0.4856 - acc: 0.7454 - val_loss: 0.3931 - val_acc: 0.8349
Epoch 10/100
24/24 [==============================] - 588s 25s/step - loss: 0.4492 - acc: 0.7822 - val_loss: 0.3387 - val_acc: 0.8629
Epoch 11/100
24/24 [==============================] - 623s 26s/step - loss: 0.4211 - acc: 0.8155 - val_loss: 0.3678 - val_acc: 0.8349
Epoch 12/100
24/24 [==============================] - 620s 26s/step - loss: 0.4332 - acc: 0.7881 - val_loss: 0.3068 - val_acc: 0.8723
Epoch 13/100
24/24 [==============================] - 643s 27s/step - loss: 0.3907 - acc: 0.8272 - val_loss: 0.3232 - val_acc: 0.8692
Epoch 14/100
24/24 [==============================] - 619s 26s/step - loss: 0.4620 - acc: 0.7600 - val_loss: 0.3639 - val_acc: 0.8505
Epoch 15/100
24/24 [==============================] - 695s 29s/step - loss: 

24/24 [==============================] - 584s 24s/step - loss: 0.2219 - acc: 0.9146 - val_loss: 0.2058 - val_acc: 0.9065
Epoch 70/100
24/24 [==============================] - 572s 24s/step - loss: 0.2246 - acc: 0.9098 - val_loss: 0.1561 - val_acc: 0.9377
Epoch 71/100
24/24 [==============================] - 583s 24s/step - loss: 0.2360 - acc: 0.9012 - val_loss: 0.2355 - val_acc: 0.9034
Epoch 72/100
24/24 [==============================] - 584s 24s/step - loss: 0.2780 - acc: 0.8640 - val_loss: 0.3057 - val_acc: 0.8816
Epoch 73/100
24/24 [==============================] - 589s 25s/step - loss: 0.2004 - acc: 0.9120 - val_loss: 0.1797 - val_acc: 0.9315
Epoch 74/100
24/24 [==============================] - 586s 24s/step - loss: 0.2083 - acc: 0.9094 - val_loss: 0.2175 - val_acc: 0.9097
Epoch 75/100
24/24 [==============================] - 585s 24s/step - loss: 0.2255 - acc: 0.9113 - val_loss: 0.1665 - val_acc: 0.9408
Epoch 76/100
24/24 [==============================] - 586s 24s/step - loss:

24/24 [==============================] - 586s 24s/step - loss: 0.8260 - acc: 0.5811 - val_loss: 0.5721 - val_acc: 0.6406
Epoch 3/100
24/24 [==============================] - 581s 24s/step - loss: 0.7949 - acc: 0.5896 - val_loss: 0.7216 - val_acc: 0.5312
Epoch 4/100
24/24 [==============================] - 582s 24s/step - loss: 0.7745 - acc: 0.5774 - val_loss: 0.6931 - val_acc: 0.4719
Epoch 5/100
24/24 [==============================] - 586s 24s/step - loss: 0.6930 - acc: 0.6141 - val_loss: 0.5677 - val_acc: 0.7781
Epoch 6/100
24/24 [==============================] - 583s 24s/step - loss: 0.6171 - acc: 0.6863 - val_loss: 0.4241 - val_acc: 0.7844
Epoch 7/100
24/24 [==============================] - 560s 23s/step - loss: 0.4967 - acc: 0.7672 - val_loss: 0.4538 - val_acc: 0.7344
Epoch 8/100
24/24 [==============================] - 581s 24s/step - loss: 0.4816 - acc: 0.7661 - val_loss: 0.4571 - val_acc: 0.7594
Epoch 9/100
24/24 [==============================] - 583s 24s/step - loss: 0.5247

24/24 [==============================] - 562s 23s/step - loss: 0.2155 - acc: 0.9164 - val_loss: 0.2559 - val_acc: 0.8938
Epoch 64/100
24/24 [==============================] - 594s 25s/step - loss: 0.1912 - acc: 0.9139 - val_loss: 0.2464 - val_acc: 0.9125
Epoch 65/100
24/24 [==============================] - 580s 24s/step - loss: 0.2164 - acc: 0.9202 - val_loss: 0.2469 - val_acc: 0.9094
Epoch 66/100
24/24 [==============================] - 580s 24s/step - loss: 0.2215 - acc: 0.8983 - val_loss: 0.2516 - val_acc: 0.9062
Epoch 67/100
24/24 [==============================] - 581s 24s/step - loss: 0.1771 - acc: 0.9361 - val_loss: 0.2153 - val_acc: 0.9219
Epoch 68/100
24/24 [==============================] - 580s 24s/step - loss: 0.2099 - acc: 0.9098 - val_loss: 0.2797 - val_acc: 0.8781
Epoch 69/100
24/24 [==============================] - 581s 24s/step - loss: 0.2067 - acc: 0.9072 - val_loss: 0.2268 - val_acc: 0.9094
Train loss: 0.174121474038
Train accuracy: 0.930685358255
Test loss: 0.2106

In [16]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv(path + 'sub.csv', index=False)